If you use this code, please cite

[Kazuki Nakajima, Yuya Sasaki, Takeaki Uno, and Masaki Aida. (2025). Learning Multi-Order Block Structure in Higher-Order Networks. arXiv preprint arXiv:2511.21350.](https://arxiv.org/abs/2511.21350)

In [1]:
import statistics
import time, pickle
import hypergraph, model_evaluation, hypermosbm

First, we prepare the hypergraph to be analyzed. We construct a hypergraph object using the `HyperGraph` class from the `hypergraph` module.

Specifically, the following four items (plus two optional items) are required:
- **N**: Number of nodes.
- **M**: Number of hyperedges.
- **E**: List of hyperedges. Each element is a list of node IDs representing a hyperedge. For example, `E = [[0, 1], [1, 2, 3]]` indicates two hyperedges: {0, 1} and {1, 2, 3}. Node IDs should be consecutive integers ranging from 0 to $N-1$.
- **A**: List of hyperedge weights. Each element is an integer ($\ge 1$). The $i$-th element corresponds to the observation count of the $i$-th hyperedge in list `E`. For example, `E = [[0, 1], [1, 2, 3]]` and `A = [1, 2]` means the hyperedge {0, 1} appears once, and {1, 2, 3} appears twice in the data.
- **Z (Optional)**: Number of unique categories in node metadata (integer $\ge 1$).
- **X (Optional)**: List of node categories. Each element represents the category ID to which a node belongs. Category IDs are integers ranging from 0 to $Z-1$.

After preparing these, use the `hypergraph.construct_hypergraph` function to build the hypergraph. An example is provided below.

In [2]:
N = 4
M = 2
E = [[0, 1], [1, 2, 3]]
A = [1, 2]
Z = 2 # Optional: Number of categories in node metadata
X = [0, 1, 0, 1] # Optional: List of node categories

H = hypergraph.construct_hypergraph(
    N=N, # Number of nodes
    M=M, # Number of hyperedges
    E=E, # Hyperedge list
    A=A, # Hyperedge-weight list
    Z=Z, # Number of categories in node metadata 
    X=X, # Node metadata list 
)

Number of nodes: 4
Number of hyperedges: 2
Sum of hyperedge weights: 3
Average degree of the node: 1.25
Average size of the hyperedge: 2.5
Maximum size of the hyperedge: 3
Hyperedge size distribution: [(2, 1), (3, 1)]
Connected hypergraph: True


We save the constructed hypergraph object as a pickle file in the `./data/` directory. You may choose any filename.

In [3]:
# Save the hypergraph at the './data/' directory
with open('./data/example_hypergraph.pickle', 'wb') as f:
    pickle.dump(H, f)

In the following examples, we use a small synthetic hypergraph generated by a hypergraph stochastic block model (SBM).  
For details on the generation procedure and parameters, please refer to Section 4.4 of our manuscript:  
[Kazuki Nakajima, Yuya Sasaki, Takeaki Uno, and Masaki Aida. (2025). Learning Multi-Order Block Structure in Higher-Order Networks. arXiv preprint arXiv:2511.21350.](https://arxiv.org/abs/2511.21350)

In [4]:
N = 100 # Number of nodes
K = 2 # Number of communities
avg_node_degree = 20 # Average node degree
beta_out_2 = 1 # probability b for pairwise interactions
alpha_in_2 = float(5.0) # probability a for pairwise interactions
lambda_div = 1.0 # heterogeneity parameter zeta (controls deviation of higher-order patterns)

# Return the tuple of a generated hypergraph and its underlying community membership matrix
(H, U_true) = hypergraph.generate_hsbm(
    N, 
    K, 
    alpha_in_2, 
    beta_out_2, 
    lambda_div, 
    avg_node_degree,
)

with open('./data/sbm_hypergraph.pickle', 'wb') as f:
        pickle.dump(H, f)

Number of nodes: 100
Number of hyperedges: 737
Sum of hyperedge weights: 737.0
Average degree of the node: 20.77
Average size of the hyperedge: 2.8181818181818183
Maximum size of the hyperedge: 4
Hyperedge size distribution: [(2, 283), (3, 305), (4, 149)]
Connected hypergraph: True


We load the constructed hypergraph.

In [5]:
H = hypergraph.read_pickle_file("sbm_hypergraph.pickle")

We construct training and testing sets for cross-validation to calculate the hyperlink prediction AUC during the search for the optimal partition of the set of hyperedge sizes (interaction orders). We specify the number of splits, `n_splits`.

In [6]:
n_splits = 10
train_and_test_sets = model_evaluation.construct_train_and_test_sets(H, n_splits)

We search for the optimal partition of the set of hyperedge sizes by providing the hypergraph `H`, the number of communities `K` (a hyperparameter), and the train-and-test sets. 
The search begins with the coarsest partition, where all hyperedge sizes form a single set (i.e., the single-order model), and iteratively proceeds to maximize the AUC. We obtain the search history and the final partition of the hyperedge sizes.

In [7]:
t_s = time.time()
res_history, final_hs_clusters = hypermosbm.partition_hyperedge_size_set(
    H=H, 
    K=K, 
    train_and_test_sets=train_and_test_sets,
)
t_e = time.time()

Exhaustive search finished. Best partition: [[2], [3, 4]] with mean AUC = 0.6003


Let's check the time required for determining the best partition.

In [8]:
print(f"Time taken for determining the best partition of the set of hyperedge sizes: {t_e - t_s}")

Time taken for determining the best partition of the set of hyperedge sizes: 118.23751401901245


`res_history` stores the search history as a list. The first element corresponds to the initial single-order partition, and the last element corresponds to the final partition.
Each entry in the history is a list of length 4 containing:
1. The partition of hyperedge sizes.
2. The list of AUCs for the 10 test sets.
3. The list of sample sizes for the 10 test sets.
4. The list of computation times for the 10 test sets.

In [9]:
# Result for the single-order partition
print(res_history[0])

[[[2, 3, 4]], [0.5636, 0.46545, 0.5103, 0.55385, 0.4924, 0.541, 0.5119, 0.48525, 0.498, 0.5343], [74, 74, 74, 74, 74, 74, 74, 73, 73, 73], [1.5257370471954346, 1.5048329830169678, 1.5019547939300537, 1.4967999458312988, 1.4982609748840332, 1.504676103591919, 1.4995839595794678, 1.5018649101257324, 1.5027971267700195, 1.5102860927581787]]


In [10]:
# Result for the final partition
print(res_history[-1])

[[[2], [3], [4]], [0.5791, 0.62085, 0.60935, 0.52195, 0.5475, 0.51955, 0.4669, 0.57985, 0.6272, 0.5861], [74, 74, 74, 74, 74, 74, 74, 73, 73, 73], [3.034085750579834, 3.0532889366149902, 3.0561931133270264, 3.044346809387207, 3.0428390502929688, 3.043346881866455, 3.0657830238342285, 3.058501958847046, 3.063037872314453, 3.050771951675415]]


`final_hs_clusters` is the finally selected partition.

In [11]:
final_hs_clusters

[[2], [3, 4]]

The mean AUC of the final partition minus that of the single partition, $\Delta_{\text{AUC}}$, is as follows.

In [12]:
statistics.mean(res_history[-1][1]) - statistics.mean(res_history[0][1])

0.05023

We are now ready to fit the HyperMOSBM to the hypergraph `H` using the number of communities `K` and the `final_partition`. 
We perform `n_init` random initializations and obtain the estimated parameters `U` and `W` corresponding to the highest log-likelihood.

In [13]:
model = hypermosbm.HyperMOSBM(H, K, hs_clusters=final_hs_clusters)
best_loglik, (U, W) = model.fit(
    n_init=10, 
    max_iter=500,
)

`U` is the estimated community membership matrix of size $N \times K$. The $i$-th row of the matrix represents the community membership vector for node ID $i$.

In [14]:
print(U)

[[6.02760289e-196 1.44706797e-001]
 [1.19589065e-001 1.70513048e-001]
 [2.40064498e-001 1.12826332e-001]
 [1.20144703e-001 3.36085606e-170]
 [3.34112491e-001 4.28015864e-002]
 [1.25809429e-001 2.04384735e-001]
 [2.69617706e-164 2.62471706e-001]
 [2.11531754e-001 1.10151417e-056]
 [2.55297591e-136 2.62559486e-001]
 [1.36657802e-001 5.94107042e-018]
 [2.63443135e-002 1.43674107e-001]
 [2.15839174e-001 5.79173465e-002]
 [3.66992012e-001 1.91060763e-204]
 [2.07077789e-001 5.14114053e-011]
 [4.29655536e-002 9.51066596e-002]
 [3.96148088e-143 1.56266489e-001]
 [1.69323112e-172 2.19658598e-001]
 [1.91280267e-154 2.68571600e-001]
 [4.50340402e-002 2.57266578e-001]
 [3.20990434e-001 7.75121406e-031]
 [3.04342365e-001 3.83678412e-078]
 [2.83098380e-001 2.91558971e-130]
 [2.67837806e-001 1.25283245e-044]
 [3.43984213e-001 6.16951709e-029]
 [3.89163797e-172 2.95845588e-001]
 [1.06666004e-001 4.38036700e-002]
 [7.54539906e-061 3.48926202e-001]
 [2.02350535e-001 1.91838719e-144]
 [2.80177061e-001 1.

`W` is a collection of $L$ affinity matrices, where $L$ is the cardinality of the final partition. 
The $l$-th element of `W` is the estimated affinity matrix corresponding to the $l$-th subset in `final_hs_clusters`. Each matrix is of size $K \times K$, where the $(k, q)$ entry represents the density of interactions between the $k$-th and $q$-th communities.

In [15]:
for l in range(0, len(final_hs_clusters)):
    print(l, W[l])

0 [[2.10058931e+000 1.74458139e-116]
 [1.74458139e-116 1.45638017e+000]]
1 [[6.08849061e-21 5.54593558e+00]
 [5.54593558e+00 6.24174692e-05]]
